In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
from theano import *
from lasagne.layers import InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np
from helpers import SimpleMaxingLayer, SimpleAverageLayer
from wordvecs import WordVectors, EmbeddingLayer
import json
import re

theano.config.floatX = 'float32'
#theano.config.linker = 'cvm_nogc'
theano.config.openmp = True
theano.config.openmp_elemwise_minsize = 20000

In [3]:
with open('/data/matthew/external-wiki2.json') as f:
    queries = json.load(f)['queries']

In [4]:
len(queries)

9915

In [5]:
sum([any([g['gold'] for g in v.values()]) for v in queries.values()])

8917

In [6]:
wordvectors = WordVectors(
    fname="/data/matthew/enwiki-20141208-pages-articles-multistream-links5-output1.bin",
    redir_fname='/data/matthew/enwiki-20141208-pages-articles-multistream-redirects5.json',
    negvectors=False,
    sentence_length=200,
)
wordvectors.add_unknown_words = False

In [7]:
# with open('/data/matthew/enwiki-20141208-pages-articles-multistream-redirects5.json') as f:
#     page_redirects = json.load(f)
page_redirects = wordvectors.redirects

In [8]:
len(wordvectors.vectors)

4613263

In [9]:
from wikireader import WikiRegexes, WikipediaReader

In [10]:
def PreProcessedQueries(wikipedia_dump_fname, wordvec=wordvectors, queries=queries, redirects=page_redirects):
    
    get_words = re.compile('[^a-zA-Z0-9 ]')
    get_link = re.compile('.*?\[(.*?)\].*?')
    
    queried_pages = set()
    for docs, q in queries.iteritems():
        wordvec.tokenize(docs, length=200)
        for sur, v in q.iteritems():
            wrds_sur = get_words.sub(' ', sur)
            wordvec.tokenize(wrds_sur)
            link_sur = get_link.match(sur).group(1)
            wordvec.tokenize(link_sur)
            for link in v['vals'].keys():
                wrds = get_words.sub(' ', link)
                wordvec.tokenize(wrds)
                tt = WikiRegexes.convertToTitle(link)
                wordvec.get_location(tt)
                queried_pages.add(tt)

    added_pages = set()
    for title in queried_pages:
        if title in redirects:
            #wordvec.tokenize(self.redirects[title])
            added_pages.add(redirects[title])
    queried_pages |= added_pages

    page_content = {}

#     class GetWikipediaWords(WikipediaReader, WikiRegexes):

#         def readPage(ss, title, content):
#             tt = ss.convertToTitle(title)
#             if tt in queried_pages:
#                 cnt = ss._wikiToText(content)
#                 page_content[tt] = wordvec.tokenize(cnt)

#     GetWikipediaWords(wikipedia_dump_fname).read()
    
    rr = redirects
    rq = queried_pages
    rc = page_content

    class PreProcessedQueriesCls(object):
        
        wordvecs = wordvec
        queries = queries
        redirects = rr
        queried_pages = rq
        page_content = rc
        
        
    return PreProcessedQueriesCls

In [11]:
basePreProcessedQueries = PreProcessedQueries('/data/matthew/enwiki-20141208-pages-articles-multistream.xml')

In [12]:
len(wordvectors.vectors)

4613263

In [56]:
class EntityVectorLinkExp(basePreProcessedQueries):
    
    batch_size = 1000 #20000
    num_training_items = 1000 #200000
    
    def __init__(self): #, wikipedia_dump_fname, wordvec=wordvectors, queries=queries, redirects=page_redirects):
        #self.wordvecs = wordvec
        #self.queries = queries
        self.sentence_length = self.wordvecs.sentence_length
        self.document_length = 100
        self.num_words_to_use_conv = 5
        #self.redirects = redirects
        #self.page_content = {}
        #self.wikipedia_dump_fname = wikipedia_dump_fname
        
        #self._process_queries()
        
        self._setup()
        
#     def _process_queries(self):
#         queried_pages = set()
#         for docs, q in self.queries.iteritems():
#             self.wordvecs.tokenize(docs)
#             for sur, v in q.iteritems():
#                 self.wordvecs.tokenize(sur)
#                 for link in v['vals'].keys():
#                     self.wordvecs.tokenize(link)
#                     tt = WikiRegexes.convertToTitle(link)
#                     #self.wordvecs.tokenize(tt)
#                     queried_pages.add(tt)

#         added_pages = set()
#         for title in queried_pages:
#             if title in self.redirects:
#                 #self.wordvecs.tokenize(self.redirects[title])
#                 added_pages.add(self.redirects[title])
#         queried_pages |= added_pages
        
#         self.queried_pages = queried_pages
                
#         class GetWikipediaWords(WikipediaReader, WikiRegexes):
            
#             def readPage(ss, title, content):
#                 tt = ss.convertToTitle(title)
#                 if tt in queried_pages:
#                     cnt = ss._wikiToText(content)
#                     self.page_content[tt] = self.wordvecs.tokenize(cnt)
        
#         GetWikipediaWords(self.wikipedia_dump_fname).read()
               
        
    def _setup(self):
        self.x_document_input = T.imatrix('x_doc')
        
        self.x_document_id = T.ivector('x_doc_id')
        self.x_surface_text_input = T.imatrix('x_surface_link')
        self.x_surface_context_input = T.imatrix('x_surface_cxt')  # TODO
        
        self.x_target_input = T.ivector('x_target')
        self.x_target_words = T.imatrix('x_target_words')
        self.x_matches_surface = T.ivector('x_match_surface')
        self.x_link_id = T.ivector('x_link_id')
        
        #self.y_score = T.vector('y')
        self.y_answer = T.ivector('y_ans')  # contains the location of the gold answer so we can compute the loss
        
        self.embedding_W = theano.shared(self.wordvecs.get_numpy_matrix().astype(theano.config.floatX))
        
        self.document_l = lasagne.layers.InputLayer(
            (None,self.document_length), 
            input_var=self.x_document_input
        )
    
        self.document_embedding_l = EmbeddingLayer(
            self.document_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.document_conv1_l = lasagne.layers.Conv2DLayer(
            self.document_embedding_l,
            num_filters=500,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='document_conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.document_max_l = lasagne.layers.Pool2DLayer(
            self.document_conv1_l,
            name='document_pool1',
            pool_size=(self.document_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )

        self.document_dens1 = lasagne.layers.DenseLayer(
            self.document_max_l,
            num_units=250,
            name='doucment_dens1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.document_drop1 = lasagne.layers.DropoutLayer(
            self.document_dens1,
            p=.25,
        )
        
        document_output_length = 200
        
        self.document_dens2 = lasagne.layers.DenseLayer(
            self.document_drop1,
            num_units=document_output_length,
            name='document_dens2',
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )
        
        self.document_output = lasagne.layers.get_output(self.document_dens2)
        
        self.surface_context_l = lasagne.layers.InputLayer(
            (None, self.sentence_length),
            input_var=self.x_surface_context_input,
        )
        
        self.surface_context_embedding_l = EmbeddingLayer(
            self.surface_context_l,
            W=self.embedding_W,
            add_word_params=False,
        )
                
        self.surface_context_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_context_embedding_l,
            num_filters=300,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_cxt_conv1',
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )
        
#         self.surface_context_avg1_l = SimpleAverageLayer(
#             [self.surface_context_conv1_l, self.surface_context_l],
#             #name='surface_context_avg'
#         )
        
        self.surface_context_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_context_conv1_l,
            name='surface_cxt_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )
                
        self.surface_input_l = lasagne.layers.InputLayer(
            (None, self.sentence_length), 
            input_var=self.x_surface_text_input
        )
        
        self.surface_embedding_l = EmbeddingLayer(
            self.surface_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.surface_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_embedding_l,
            num_filters=300,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_conv1',
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )
        
#         self.surface_avg1_l = SimpleAverageLayer(
#             [self.surface_conv1_l, self.surface_input_l],
#             #name='surface_avg'
#         )
        
        self.surface_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_conv1_l,
            name='surface_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )
        
        self.surface_merged_l = lasagne.layers.ConcatLayer(
            [self.surface_context_pool1_l, self.surface_pool1_l]
        )
        
        self.surface_dens1 = lasagne.layers.DenseLayer(
            self.surface_merged_l,
            name='surface_dens1',
            num_units=250,
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )
        
#         self.surface_drop1 = lasagne.layers.DropoutLayer(
#             self.surface_dens1,
#             p=.25,
#         )
        
#         self.surface_dens2 = lasagne.layers.DenseLayer(
#             self.surface_drop1,
#             name='surface_dens2',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )
        
        self.document_aligned_l = InputLayer(
            (None, document_output_length),
            input_var=self.document_output[self.x_document_id,:]
        )
        
        self.source_l = lasagne.layers.ConcatLayer(
            [self.document_aligned_l, self.surface_dens1]
        )
        
        self.source_dens1 = lasagne.layers.DenseLayer(
            self.source_l,
            num_units=300,
            name='source_dens1',
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )
        
        self.source_drop1 = lasagne.layers.DropoutLayer(
            self.source_dens1,
            p=.25,
        )
        
        self.source_dens12 = lasagne.layers.DenseLayer(
            self.source_drop1,
            num_units=250,
            name='source_dens12',
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )
        
        self.source_drop12 = lasagne.layers.DropoutLayer(
            self.source_dens12,
            p=.25,
        )
        
        compared_vector_size = self.wordvecs.vector_size #+ 2 # extra space for if it matches the surface text
        
        self.source_dens2 = lasagne.layers.DenseLayer(
            self.source_drop12,
            num_units=compared_vector_size,  # this is the same size as the learned wikipedia vectors
            name='source_dens2',
            nonlinearity=lasagne.nonlinearities.linear,
        )
        
        self.source_out = lasagne.layers.get_output(self.source_dens2)
        
        self.target_input_l = lasagne.layers.InputLayer(
            (None,),
            input_var=self.x_target_input
        )
        
        self.target_embedding_l = EmbeddingLayer(
            lasagne.layers.reshape(self.target_input_l, ([0], 1)),
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.target_words_input_l = lasagne.layers.InputLayer(
            (None,self.sentence_length),
            input_var=self.x_target_words,
        )
        
        self.target_words_embedding_l = EmbeddingLayer(
            self.target_words_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.target_words_conv1_l = lasagne.layers.Conv2DLayer(
            self.target_words_embedding_l,
            name='target_wrds_conv1',
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            num_filters=350,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.target_words_pool1_l = lasagne.layers.Pool2DLayer(
            self.target_words_conv1_l,
            name='target_wrds_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )
        
        self.target_merge_l = lasagne.layers.ConcatLayer(
            [lasagne.layers.reshape(self.target_words_pool1_l, ([0], [1])),
             lasagne.layers.reshape(self.target_embedding_l, ([0], [3]))]
        )
        
        self.target_dens1 = lasagne.layers.DenseLayer(
            self.target_merge_l,
            name='target_wrds_dens1',
            num_units=400,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.target_drop1 = lasagne.layers.DropoutLayer(
            self.target_dens1,
            p=.25,
        )
        
        self.target_dens2 = lasagne.layers.DenseLayer(
            self.target_drop1,
            name='target_wrds_dens1',
            num_units=compared_vector_size,
            nonlinearity=lasagne.nonlinearities.linear,
        )
        
#         self.target_dens1 = lasagne.layers.DenseLayer(
#             self.target_conv1_l,
#             name='target_dens1',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )
        
#         self.target_drop1 = lasagne.layers.DropoutLayer(
#             self.target_dens1,
#             p=.25,
#         )
        
#         self.target_dens2 = lasagne.layers.DenseLayer(
#             self.target_drop1,
#             name='target_dens2',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )
        
        
    
        #self.target_out = lasagne.layers.get_output(self.target_embedding_l)
        
        matched_surface_reshaped = self.x_matches_surface.reshape((self.x_matches_surface.shape[0], 1)).astype(theano.config.floatX)
        
#         self.target_out = T.concatenate(
#             [self.embedding_W[self.x_target_input], 
#              matched_surface_reshaped,
#             1-matched_surface_reshaped],
#              axis=1)
        
        self.target_out = self.embedding_W[self.x_target_input]
        #self.target_out = lasagne.layers.get_output(self.target_dens2)
        
        # compute the cosine distance between the two layers
        self.source_aligned_l = self.source_out[self.x_link_id, :]
        
        # this uses scan internally, which means that it comes back into python code to run the loop.....fml
        self.dotted_vectors =  T.batched_dot(self.target_out, self.source_aligned_l)
        # diag also does not support a C version.........
        #self.dotted_vectors = T.dot(self.target_out, self.source_aligned_l.T).diagonal()
        
        def augNorm(v):
            return T.maximum(T.basic.pow(T.basic.pow(T.basic.abs_(v), 2).sum(axis=1) + .001, .5), .001)
    
        self.res_l = self.dotted_vectors / (augNorm(self.target_out) * augNorm(self.source_aligned_l) + .001)
#         self.res_l = self.dotted_vectors / ((self.target_out.norm(1, axis=1) + .001) * 
#                                             (self.source_aligned_l.norm(1, axis=1) + .001))
        
        self.res_cap = T.clip((T.tanh(self.res_l) + 1) / 2, .001, .999)
    
        self.all_params = (
            #lasagne.layers.get_all_params(self.target_dens2) +
            # TODO: add params for the target stuff, 
            lasagne.layers.get_all_params(self.source_dens2) +
            lasagne.layers.get_all_params(self.document_dens2)
        )
        
        # weight the positive samples more since there are fewer of them,
        # freaking hack
        #self.loss_vec = -(10 * self.y_score * T.log(self.res_cap) + (1.0 - self.y_score) * T.log(1.0 - self.res_cap))
        
        #self.loss_vec = T.nnet.binary_crossentropy(self.res_cap, self.y_score)
        
        #self.loss_vec = T.exp(T.max(self.res_cap - self.res_cap[self.y_answer] + .1, 0)) - 1  # TODO: maybe have some squared term here or something?
        #self.loss_vec = - T.log((T.clip(self.res_cap[self.y_answer] - self.res_cap, -1.0, 0.4) + 1.0) / 1.5)
        
        self.loss_vec = - T.log((T.clip(self.res_l[self.y_answer] - self.res_l, -40.0, 10.0) + 40.0) / 51.0)
        
        self.updates = lasagne.updates.adadelta(self.loss_vec.mean(), self.all_params)
        
        self.func_inputs = [
            self.x_document_input,
            self.x_surface_text_input, self.x_surface_context_input, self.x_document_id,
             self.x_target_input, self.x_matches_surface, self.x_link_id, self.y_answer]  # self.x_target_words,
        
        self.train_func = theano.function(
            self.func_inputs,
            [self.res_cap, self.loss_vec.sum(), self.loss_vec],
            updates=self.updates,
            on_unused_input='ignore',
        )
        
        self.test_func = theano.function(
            self.func_inputs,
            [self.res_cap, self.loss_vec.sum(), self.loss_vec],
            on_unused_input='ignore',
        )
        
    def reset_accums(self):
        self.current_documents = []
        self.current_surface_context = []
        self.current_surface_link = []
        self.current_link_id = []
        self.current_target_input = []
        self.current_target_words = []
        self.current_target_matches_surface = []
        self.current_target_id = []
        self.current_target_goal = []
        self.learning_targets = []
        
    def compute_batch(self, isTraining=True):
        if isTraining:
            func = self.train_func
        else:
            func = self.test_func
        self.reset_accums()
        self.total_links = 0
        self.total_loss = 0.0
        
        get_words = re.compile('[^a-zA-Z0-9 ]')
        get_link = re.compile('.*?\[(.*?)\].*?')
        
        for doc, queries in self.queries.iteritems():
            # skip the testing documents while training and vice versa
            if queries.values()[0]['training'] is not isTraining:
                continue
            docid = len(self.current_documents)
            self.current_documents.append(self.wordvecs.tokenize(doc, length=self.document_length))
            for surtxt, targets in queries.iteritems():
                self.current_link_id.append(docid)
                surid = len(self.current_surface_link)
                self.current_surface_context.append(self.wordvecs.tokenize(get_words.sub(' ' , surtxt)))
                surlink = get_link.match(surtxt).group(1)
                self.current_surface_link.append(self.wordvecs.tokenize(surlink))
                surmatch = surlink.lower()
                #target_page_input = []
                target_words_input = []
                target_matches_surface = []
                target_inputs = []
                target_learings = []
                target_gold_loc = None
                for target in targets['vals'].keys():
                    # skip the items that we don't know the gold for
                    if not targets['gold'] and isTraining:
                        continue
                    isGold = target == targets['gold']
                    #cnt = self.page_content.get(WikiRegexes.convertToTitle(target))
                    cnt = self.wordvecs.get_location(WikiRegexes.convertToTitle(target))
                    if cnt is None:
                        # were not able to find this wikipedia document
                        # so just ignore tihs result since trying to train on it will cause
                        # issues
                        continue
                    if isGold:
                        target_gold_loc = len(target_inputs)
                    #target_page_input.append(cnt)
                    target_words_input.append(self.wordvecs.tokenize(get_words.sub(' ', target)))
                    target_inputs.append(cnt)  # page_content already tokenized
                    target_matches_surface.append(int(surmatch == target.lower()))
                    target_learings.append((targets, target))
                if target_gold_loc is not None:  # if we can't get the gold item
                    # contain the index of the gold item for these items, so it can be less then it
                    self.current_target_goal += [(len(self.current_target_goal) + target_gold_loc)] * len(target_inputs)
                    self.current_target_input += target_inputs
                    self.current_target_id += [surid] * len(target_inputs)
                    self.current_target_words += target_words_input   # TODO: add
                    self.current_target_matches_surface += target_matches_surface
                
                #self.current_target_goal.append(isGold)
                self.learning_targets += target_learings
            if len(self.current_target_id) > self.batch_size:
                return
                self.run_batch(func)
                if self.total_links > self.num_training_items:
                    return self.total_loss / self.total_links
        
        if len(self.current_target_id) > 0:
            self.run_batch(func)
            
        return self.total_loss / self.total_links
        
    def run_batch(self, func):
        res_vec, loss_sum, loss_vec = func(
            self.current_documents,
            self.current_surface_link, self.current_surface_context, self.current_link_id,
            self.current_target_input, self.current_target_matches_surface, self.current_target_id, self.current_target_goal  # self.current_target_words,
        )
        self.check_params()
        self.total_links += len(self.current_target_id)
        self.total_loss += loss_sum
        for i in xrange(len(res_vec)):
            # save the results from this pass
            l = self.learning_targets[i]
            l[0]['vals'][ l[1] ] = res_vec[i]
        self.reset_accums()
        
    def check_params(self):
        if any([np.isnan(v.get_value(borrow=True)).any() for v in self.all_params]):
            raise RuntimeError('nan in some of the parameters')
        

        
queries_exp = EntityVectorLinkExp() 

In [57]:
queries_exp.check_params()

In [58]:
%time print queries_exp.compute_batch()

None
CPU times: user 51 ms, sys: 7.05 ms, total: 58.1 ms
Wall time: 43.3 ms


In [25]:
exp_results = []

for i in xrange(20):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)

(0, 0.24303590798679786)
(1, 0.24288906628572488)
(2, 0.24267305301714548)
(3, 0.24254185640359227)


KeyboardInterrupt: 

In [ ]:
for i in xrange(20):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)

In [40]:
len(queries_exp.current_target_goal), len(queries_exp.current_target_inputa)

(256091, 256091)

In [ ]:
queries_exp.compute_batch(False)

In [73]:
exp_results

[(0, 1.2287285061402835e-05), (1, 1.1753841393782684e-05)]

In [129]:
all_measured = 0
all_correct = 0
for qu in queries.values():
    for en in qu.values():
        for e in en:
            if en['gold']:
                if all_measured > 20000:
                    break
                all_measured += 1
                m = max(en['vals'].values())
                if en['vals'][en['gold']] == m:
                    all_correct += 1

In [130]:
all_measured

20001

In [131]:
float(all_correct) / all_measured

0.14069296535173242

In [133]:
queries.values()[10]

{u"'' Saint Jerome in his Study '' (1514) and '' [Melencolia I] '' (1514) , which has been the subject of extensive": {u'gold': u'Melencolia I',
  u'training': True,
  u'vals': {u'-NIL-': 0.55039364,
   u'Belarusian Premier League': 0.59051669,
   u'Division I (NCAA)': 0.59985805,
   u'I': 0.52737033,
   u'I (band)': 0,
   u'I (film)': 0.53069359,
   u'I Corps (German Empire)': 0,
   u'Independent (politician)': 0.59051669,
   u'Independent politician': 0.57096189,
   u'Instrumental case': 0.52732587,
   u'Iodine': 0.45036077,
   u'Isospin': 0.50938243,
   u'Italy': 0.51380736,
   u'Meistriliiga': 0.50938243,
   u'Melencolia I': 0.506374,
   u'Nemzeti Bajnoks\xe1g I': 0.50964814,
   u'Russian Football Premier League': 0.49646947,
   u'Russian Premier League': 0.4457525,
   u'Super Bowl I': 0.59305191,
   u'XXNILXX': 0}},
 u"'' ndash ; April 6 , 1528 -RRB- was a [German] painter , printmaker and theorist from Nuremberg , Germany .": {u'gold': u'Germans',
  u'training': True,
  u'vals': 

In [142]:
queries_exp.compute_batch()

In [59]:
gg_func = theano.function(
    queries_exp.func_inputs,
    [queries_exp.loss_vec, queries_exp.source_out, queries_exp.target_out] + T.grad(queries_exp.loss_vec.mean(), queries_exp.all_params),
    updates=queries_exp.updates,
    on_unused_input='ignore',
)

In [80]:
res1 = gg_func(
    queries_exp.current_documents,
    queries_exp.current_surface_link, queries_exp.current_surface_context, queries_exp.current_link_id,
    queries_exp.current_target_input, queries_exp.current_target_matches_surface, queries_exp.current_target_id, queries_exp.current_target_goal
)

In [69]:
res1[0].max(), res1[0].mean(), res1[0].min()

(0.24727535, 0.24219072, 0.23549788)

In [81]:
np.dot(res1[1][0], res1[2][0:10,:].T)

array([  76.69731903,  172.44116211,  139.30476379, -182.91343689,
        172.44116211,  172.44116211,  -13.79522324,   97.6740036 ,
         11.84405231,   36.98944092], dtype=float32)

In [83]:
res1[0][0:10]

array([ 0.24216416,  0.24294616,  0.24274334,  0.23833458,  0.24294616,
        0.24294616,  0.24024701,  0.24193516,  0.24127102,  0.24146625], dtype=float32)

In [71]:
res1[3:][0].std()

9.7740622e-06

In [53]:
queries_exp.all_params

[surface_cxt_conv1.W,
 surface_cxt_conv1.b,
 surface_conv1.W,
 surface_conv1.b,
 surface_dens1.W,
 surface_dens1.b,
 source_dens1.W,
 source_dens1.b,
 source_dens12.W,
 source_dens12.b,
 source_dens2.W,
 source_dens2.b,
 document_conv1.W,
 document_conv1.b,
 doucment_dens1.W,
 doucment_dens1.b,
 document_dens2.W,
 document_dens2.b]

In [197]:
res1[0].mean()

0.24253285

In [198]:
np.dot(res1[2][0,:],res1[1][0:15,:].T)

array([-10.04426765,  -8.85146523,  -6.5927825 ,  -5.64352655,
        -6.2810235 , -10.34654236,  -6.09899616,   1.8380394 ,
        -3.23892975,  -6.04222679,  -8.91859531,   1.09592962,
        -5.27822304, -12.33982086,  -5.90433264], dtype=float32)

In [178]:
res1[2][0,:].min()

-5.4103508

In [168]:
res1[1][0,:]

-0.82824624

In [ ]:
np.isnan(gg_res[0]).any()

In [ ]:
gg_res[2].mean()

In [ ]:
gg_res[0].mean()

In [ ]:
[(v, np.isnan(v.get_value(borrow=True)).all()) for v in queries_exp.all_params]

In [ ]:
queries_exp.total_links

In [ ]:
theano.printing.pydotprint(T.grad(queries_exp.loss_vec.mean(), queries_exp.all_params))

In [ ]:
gg_func = theano.function(
            [queries_exp.x_document_input,
             queries_exp.x_surface_text_input, queries_exp.x_document_id,
             queries_exp.x_target_input, queries_exp.x_link_id, queries_exp.y_score],
            T.grad(queries_exp.loss_vec.mean(), lasagne.layers.get_all_params(queries_exp.target_dens2)),
#           T.grad(queries_exp.loss_vec.mean(), queries_exp.all_params),
    #             [queries_exp.target_out, queries_exp.source_aligned_l, 
#              T.dot(queries_exp.target_out, queries_exp.source_aligned_l.T).diagonal(),
#              queries_exp.target_out.norm(2, axis=1) * queries_exp.source_aligned_l.norm(2, axis=1),
#              T.batched_dot(queries_exp.target_out, queries_exp.source_aligned_l),
#              lasagne.layers.get_output(queries_exp.target_dens2),
#              queries_exp.target_out.norm(2, axis=1),
#              #T.grad(queries_exp.loss_vec.mean(), queries_exp.all_params)
#             ],
        #[queries_exp.res_l, queries_exp.loss_vec.sum(), queries_exp.loss_vec],
    on_unused_input='ignore',
    mode='DebugMode'
        )

In [19]:
gg_func = theano.function(
            [queries_exp.x_document_input,
             queries_exp.x_surface_text_input, queries_exp.x_document_id,
             queries_exp.x_target_input, queries_exp.x_link_id, queries_exp.y_answer],
            #[self.res_cap, self.loss_vec.sum(), self.loss_vec],
            [queries_exp.res_cap],
        mode='DebugMode',
        on_unused_input='ignore',
        )

In [20]:
try:
    gg_grad_res = gg_func(
        queries_exp.current_documents,
        queries_exp.current_surface_text, queries_exp.current_link_id,
        queries_exp.current_target_input, queries_exp.current_target_id, queries_exp.current_target_goal
    )
except Exception as e:
    eeee = e

In [21]:
eeee

ValueError('Bad input argument to theano function with name "<ipython-input-19-c6e818cc55a5>:8"  at index 4(0-based)',
           'setting an array element with a sequence.')

In [ ]:
[np.isnan(v).any() for v in gg_grad_res]

In [ ]:
eeee

In [ ]:
len(queries_exp.current_target_input)

In [ ]:
gg_res = gg_func(
    queries_exp.current_documents,
    queries_exp.current_surface_text, queries_exp.current_link_id,
    queries_exp.current_target_input, queries_exp.current_target_id, queries_exp.current_target_goal
)

In [ ]:
len(queries_exp.current_target_input)

In [ ]:
np.isnan(gg_res[5]).any()

In [ ]:
gg_res[2]

In [ ]:
gg_res[0].shape, gg_res[1].shape, gg_res[2].shape, gg_res[3].shape, gg_res[4].shape

In [ ]:
np.inner(gg_res[0], gg_res[1]).shape

In [ ]:
gg_res[0] * gg_res[1]

In [ ]:
aa = np.dot(gg_res[0], gg_res[1].T)

In [ ]:
aa.diagonal().shape

In [ ]:
gg_res[2]

In [ ]:
len(queries_exp.queried_pages)

In [ ]:
len(queries_exp.page_content)

In [ ]:
len(queries_exp.current_surface_text)

In [ ]:
len(queries_exp.current_link_id)

In [ ]:
len(queries_exp.current_target_input)

In [ ]:
exp_results = []

for i in xrange(5):
    exp_results.append((i, queries_exp.compute_batch()))

In [ ]:
exp_results

In [ ]:
queries.values()

In [ ]:
queries_exp.total_links

In [ ]:
queries_exp.total_loss / queries_exp.total_links

In [ ]:
queries_exp.total_loss

In [ ]:
len(queries_exp.current_target_id)

In [ ]:
queries_exp.compute_batch()

In [ ]:
%time queries_exp.run_batch(queries_exp.test_func)